<a href="https://colab.research.google.com/github/taabata/BMAIL/blob/main/StableDiffusion_diffusers_quickly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Minimal notebook for running (and re-running) StableDiffusion via 🧨diffusers on Google Colab.

Sign-up for a [HuggingFace](https://huggingface.co/join) account.

See the [StableDiffusion model card (v1-4)](https://huggingface.co/CompVis/stable-diffusion-v1-4) for usage information before starting.

_Notebook by [Mark Adams](https://latent.differentialist.info)_

## GPU

Check that the GPU is active. If not, go to the menu and set _Runtime > Change Runtime Type > Hardware Accelerator:_ GPU.

In [ ]:
!nvidia-smi

## Connect to Google Drive

This will be used to cache models and credentials. Caching the model requires about 5GB of space on Google Drive but will save you a few minutes every time you restart the notebook. 

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!mkdir -p /content/drive/MyDrive/AI/CompVis/huggingface
!ln -fs /content/drive/MyDrive/AI/CompVis/huggingface /root/.huggingface
!mkdir -p /content/drive/MyDrive/AI/CompVis/cache
!ln -fs /content/drive/MyDrive/AI/CompVis/cache /root/.cache/huggingface

Mounted at /content/drive


Set-up for third party Jupyter widgets.

In [2]:
from google.colab import output
output.enable_custom_widget_manager()

## Install diffusers

In [3]:
pip install --upgrade diffusers transformers scipy ftfy spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 112 kB 30.3 MB/s 
     |████████████████████████████████| 4.7 MB 49.7 MB/s 
     |████████████████████████████████| 53 kB 1.9 MB/s 
     |████████████████████████████████| 101 kB 10.3 MB/s 
     |████████████████████████████████| 6.6 MB 61.1 MB/s 


## Connect to HuggingFace

Go to [Access Tokens](https://huggingface.co/settings/tokens) and create a new "User Access Token". Run the cell below and paste your token in. 

In [10]:
from os.path import exists

if not exists('/root/.huggingface/token'):
  !huggingface-cli login



        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/tokens .
        
Token: 
Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your termin

Veryify login:

In [11]:
!huggingface-cli whoami

TaabataSharran


This should return your HuggingFace name and the CompVis org. If it doesn't or you are not logged in:

- check the [model card](https://huggingface.co/CompVis/stable-diffusion-v1-4) to ensure you've got access set up.
- delete `/content/drive/MyDrive/AI/CompVis/huggingface/token` and run the login cell again.

## Set up a StableDiffusion pipeline

This will take a few minutes to complete the first time but only about a minute the next time the notebook is run.

In [12]:
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline

model_id = "CompVis/stable-diffusion-v1-4"
device = "cuda"

pipe = StableDiffusionPipeline.from_pretrained(model_id, use_auth_token=True)
pipe = pipe.to(device)

Downloading:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.4k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/342 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/543 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/743 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/71.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/522 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/335M [00:00<?, ?B/s]

## Generate images

In [84]:
from PIL import Image

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

    

In [118]:
import os
os.mkdir(os.path.join("/usr/local", "results"))

In [ ]:
import random
prompt = "a cartoon of a fuzzy orange cat sitting on planet eart, a poster by Studio Ghibli, featured on deviantart, dau-al-set, official art, toonami, anime"
seeds = [random.randint(0,100000000),55593509]
seed = seeds[0]

generator = torch.Generator(device=device).manual_seed(seed)
with autocast("cuda"):
  image = pipe(prompt,num_inference_steps=150, generator=generator,guidance_scale = 15.0 )["sample"][0]  # image here is in PIL format
  #image = pipe(prompt,num_inference_steps=15,guidance_scale = 15.0)["sample"][0]  # image here is in PIL format
image
image.save(f"/usr/local/results/{prompt}-{seed}.png")
from google.colab import files
files.download(f"/usr/local/results/{prompt}-{seed}.png")
#os.remove(f"/usr/local/{prompt}-{seed}.png")





0it [00:00, ?it/s]

In [140]:

 
dir = '/usr/local/results'
for f in os.listdir(dir):
    os.remove(os.path.join(dir, f))

## Learning more

See the 🧨[diffuser library page](https://github.com/huggingface/diffusers) the [official StableDiffusion notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/stable_diffusion.ipynb).